### Notebook for concept detection in neural network

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.capture_stones

CONCEPT_NAME = concepts.capture_stones.__name__

CASES_TO_SAMPLE = 2500 # 25000


2023-11-01 14:58:38.263832: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [3]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2
    alpha = 0.5 # The higher, the more unique games. 0 = argmax, 1 = distribution 

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state, alpha=alpha)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   0%|          | 0/2500 [00:00<?, ?it/s]

Positive cases: 2550it [03:59,  9.78it/s]                          

In [4]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2500, 4, 5, 5)


In [5]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

Positive cases: 2550it [04:10,  9.78it/s]

In [6]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  304


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [7]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [8]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = concepts.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9029 - val_loss: 0.8672
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8684 - val_loss: 0.8391
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8407 - val_loss: 0.8166
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8182 - val_loss: 0.7984
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7997 - val_loss: 0.7836
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7846 - val_loss: 0.7716
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7722 - val_loss: 0.7617
Epoch 8/50
125/125 [==============================] - 0s 2ms/step - loss: 0.7618 - val_loss: 0.7535
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7531 - val_loss: 0.7466
Epoch 10/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7458 - val_loss: 0.7406

In [9]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050, 1200, 1500, 1800]

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 271.73it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9828 - val_loss: 0.9556
Epoch 2/50
125/125 [==============================] - 0s 953us/step - loss: 0.9270 - val_loss: 0.9067
Epoch 3/50
125/125 [==============================] - 0s 945us/step - loss: 0.8809 - val_loss: 0.8647
Epoch 4/50
125/125 [==============================] - 0s 947us/step - loss: 0.8415 - val_loss: 0.8287
Epoch 5/50
125/125 [==============================] - 0s 975us/step - loss: 0.8075 - val_loss: 0.7971
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7780 - val_loss: 0.7705
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7537 - val_loss: 0.7486
Epoch 8/50
125/125 [==============================] - 0s 980us/step - loss: 0.7338 - val_loss: 0.7311
Epoch 9/50
125/125 [==============================] - 0s 945us/step - loss: 0.7186 - val_loss: 0.7181
Epoch 10/50
125/125 [==============================] -

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 288.01it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9899 - val_loss: 0.9571
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.9263 - val_loss: 0.9199
Epoch 3/50
125/125 [==============================] - 0s 928us/step - loss: 0.8887 - val_loss: 0.8919
Epoch 4/50
125/125 [==============================] - 0s 961us/step - loss: 0.8585 - val_loss: 0.8664
Epoch 5/50
125/125 [==============================] - 0s 938us/step - loss: 0.8319 - val_loss: 0.8430
Epoch 6/50
125/125 [==============================] - 0s 939us/step - loss: 0.8076 - val_loss: 0.8212
Epoch 7/50
125/125 [==============================] - 0s 928us/step - loss: 0.7852 - val_loss: 0.8010
Epoch 8/50
125/125 [==============================] - 0s 935us/step - loss: 0.7649 - val_loss: 0.7817
Epoch 9/50
125/125 [==============================] - 0s 946us/step - loss: 0.7461 - val_loss: 0.7648
Epoch 10/50
125/125 [==============================]

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 289.11it/s]


...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 4ms/step - loss: 1.0077 - val_loss: 0.9456
Epoch 2/50
125/125 [==============================] - 0s 958us/step - loss: 0.9239 - val_loss: 0.8910
Epoch 3/50
125/125 [==============================] - 0s 950us/step - loss: 0.8779 - val_loss: 0.8540
Epoch 4/50
125/125 [==============================] - 0s 959us/step - loss: 0.8433 - val_loss: 0.8239
Epoch 5/50
125/125 [==============================] - 0s 926us/step - loss: 0.8143 - val_loss: 0.7972
Epoch 6/50
125/125 [==============================] - 0s 920us/step - loss: 0.7896 - val_loss: 0.7746
Epoch 7/50
125/125 [==============================] - 0s 940us/step - loss: 0.7679 - val_loss: 0.7540
Epoch 8/50
125/125 [==============================] - 0s 970us/step - loss: 0.7483 - val_loss: 0.7355
Epoch 9/50
125/125 [==============================] - 0s 943us/step - loss: 0.7309 - val_loss: 0.7188
Epoch 10/50
125/125 [=============================

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 288.29it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9859 - val_loss: 0.9298
Epoch 2/50
125/125 [==============================] - 0s 938us/step - loss: 0.9027 - val_loss: 0.8725
Epoch 3/50
125/125 [==============================] - 0s 998us/step - loss: 0.8542 - val_loss: 0.8367
Epoch 4/50
125/125 [==============================] - 0s 926us/step - loss: 0.8196 - val_loss: 0.8062
Epoch 5/50
125/125 [==============================] - 0s 959us/step - loss: 0.7933 - val_loss: 0.7829
Epoch 6/50
125/125 [==============================] - 0s 937us/step - loss: 0.7707 - val_loss: 0.7666
Epoch 7/50
125/125 [==============================] - 0s 949us/step - loss: 0.7507 - val_loss: 0.7467
Epoch 8/50
125/125 [==============================] - 0s 929us/step - loss: 0.7326 - val_loss: 0.7300
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7162 - val_loss: 0.7153
Epoch 10/50
125/125 [==============================]

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 255.72it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9897 - val_loss: 0.9201
Epoch 2/50
125/125 [==============================] - 0s 967us/step - loss: 0.8959 - val_loss: 0.8620
Epoch 3/50
125/125 [==============================] - 0s 953us/step - loss: 0.8416 - val_loss: 0.8251
Epoch 4/50
125/125 [==============================] - 0s 933us/step - loss: 0.8049 - val_loss: 0.7984
Epoch 5/50
125/125 [==============================] - 0s 948us/step - loss: 0.7774 - val_loss: 0.7780
Epoch 6/50
125/125 [==============================] - 0s 975us/step - loss: 0.7554 - val_loss: 0.7607
Epoch 7/50
125/125 [==============================] - 0s 981us/step - loss: 0.7359 - val_loss: 0.7448
Epoch 8/50
125/125 [==============================] - 0s 968us/step - loss: 0.7195 - val_loss: 0.7308
Epoch 9/50
125/125 [==============================] - 0s 951us/step - loss: 0.7043 - val_loss: 0.7180
Epoch 10/50
125/125 [=============================